In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from util import get_data, plot_data

In [2]:
orders_file = "./orders-01.csv"
start_val = 1000000

In [137]:
# read the csv 
orders = pd.read_csv(orders_file)
dates = [dt.datetime.strptime(x, '%Y-%m-%d') for x in orders.Date.values]
orders.Date = dates

In [138]:
# set buy column
buy = [True if x == 'BUY' else False for x in orders.Order.values]
orders.Order = buy

In [139]:
end = orders.Date.max()
start = orders.Date.min()
setted = {x for x in orders.Symbol.values}
setted = list(setted)
setted

['AAPL']

In [140]:
adj_close = get_data(setted, pd.date_range(start, end))
adj_close = adj_close[setted]


In [141]:
date_list = adj_close.index
date_list.strftime('%Y-%m-%d')
date_list

DatetimeIndex(['2011-01-05'], dtype='datetime64[ns]', freq='D')

In [142]:
orders.sort_values(["Date"], inplace=True)
orders.reset_index(drop=True, inplace=True)

In [143]:
comm = 9.95
imp = 0.005
ret_df = pd.DataFrame(columns=["Portfolio Value"], index=date_list)
emp = np.zeros(len(date_list))
adj_ind = 0
order_ind = 0
order_len = orders.index.max()+1
start_val = 1000000
my_port = Portfolio(start_val, {}, comm, imp)
while adj_ind < len(date_list):
    
    while order_ind < order_len and orders.Date[order_ind] == adj_close.index[adj_ind]:
        
        sym = orders.Symbol.iloc[order_ind]
        dat = date_list[adj_ind]
        price = adj_close[sym].loc[dat]
        buy = orders.Order.iloc[order_ind]
        num = orders.Shares.iloc[order_ind]
        print sym, dat, price, buy, num
        if buy:
            my_port.buy_stock(sym, price, num)
        else:
            my_port.sell_stock(sym, price, num)
    
        order_ind += 1
    
    
    
    
    
    emp[adj_ind] = my_port.port_value(adj_close.iloc[adj_ind])
    adj_ind += 1

ret_df["Portfolio Value"] = emp
print ret_df
    

AAPL 2011-01-05 00:00:00 332.57 True 1500
AAPL 2011-01-05 00:00:00 332.57 False 1500
            Portfolio Value
2011-01-05        994991.55


In [144]:
class Portfolio:
    
    def __init__(self, cash=0, a={}, comm=0, imp=0):
        self.cash = cash
        self.stocks = a
        self.commission = comm
        self.impact = imp
        
    def buy_stock(self, stock_name, price, number_of_shares):
        # if I already have it
        if stock_name in self.stocks:
            
            # if I am shorting
            if self.stocks[stock_name][0] < 0:
                shorted = self.stocks[stock_name][0] * -1
                avg = self.stocks[stock_name][1]
                
                # if I buy less than shorted
                if number_of_shares <= shorted:          
                    self.cash += number_of_shares * avg
                    self.cash -= number_of_shares * price
                    self.stocks[stock_name][0] = -1 * shorted + number_of_shares
                    
                    # if I have exactly 0 share delete it
                    if self.stocks[stock_name][0] == 0:
                        del self.stocks[stock_name]
                
                # if I buy more than shorted
                elif number_of_shares > shorted:
                    self.cash += shorted * avg
                    self.cash -= shorted * price
                    # i will have more than 0 
                    leftover = number_of_shares - shorted
                    self.cash -= leftover * price
                    self.stocks[stock_name] = [leftover, price]
                    
            # if I am longing
            elif self.stocks[stock_name][0] > 0:
                longed = self.stocks[stock_name][0]
                avg = self.stocks[stock_name][1]
                new_avg = (longed * avg + price * number_of_shares)/(longed+number_of_shares)
                self.cash -= number_of_shares * price
                self.stocks[stock_name] = [longed + number_of_shares, new_avg]   
                
        # if I don't have it
        else:
            self.stocks[stock_name] = [number_of_shares, price]
            self.cash -= number_of_shares * price
        
        self.cash -= self.commission
        self.cash -= self.impact * number_of_shares * price
            
    def sell_stock(self, stock_name, price, number_of_shares):
        # if I already have it
        if stock_name in self.stocks:
            
            # if I am longing
            if self.stocks[stock_name][0] > 0:
                longed = self.stocks[stock_name][0]
                avg = self.stocks[stock_name][1]
                
                # if I sell less than longed
                if number_of_shares <= longed:                
                    self.cash += number_of_shares * price
                    self.stocks[stock_name] = [longed - number_of_shares, avg]
                    
                    # if I have exactly 0 share delete it
                    if self.stocks[stock_name][0] == 0:
                        del self.stocks[stock_name]
                
                # if I sell more than longed
                elif number_of_shares > longed:
                    self.cash += longed * price
                    # i will have more than 0 I am shorting here
                    leftover = number_of_shares - longed
                    self.stocks[stock_name] = [-leftover, price]
                    
            # if I am shorting
            elif self.stocks[stock_name][0] < 0:
                shorted = self.stocks[stock_name][0] 
                avg = self.stocks[stock_name][1]
                new_avg = (-1*shorted*avg + price*number_of_shares)/(-1*shorted+number_of_shares)
                self.stocks[stock_name] = [shorted - number_of_shares,  new_avg]  
                
        # if I don't have it
        else:
            self.stocks[stock_name] = [-1 * number_of_shares, price] 
            
        self.cash -= self.commission
        self.cash -= self.impact * number_of_shares * price
    
    def port_value(self, adj_close):
        val = self.cash
        for stock in self.stocks:
            if self.stocks[stock][0] > 0:
                price = adj_close[stock]
                val += price * self.stocks[stock][0]
            else:
                val += self.stocks[stock][0] * self.stocks[stock][1]
                val -= self.stocks[stock][0] * adj_close[stock]
        return val
            
    

In [80]:
a = {'b': (3, 200.0)}

In [81]:
a['b'][0]

3

In [83]:
for x in a:
    print a[x]

(3, 200.0)
